In [1]:
import os

from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole

load_dotenv()

AGENT_PROJECT_ENDPOINT = os.getenv("AGENT_PROJECT_ENDPOINT")
AGENT_MODEL_DEPLOYMENT = os.getenv("AGENT_MODEL_DEPLOYMENT")

agent_client = AgentsClient(
    endpoint=AGENT_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential
        (exclude_environment_credential=True,
         exclude_managed_identity_credential=True)
)

In [4]:
for agent in agent_client.list_agents():
    print(agent.name)

In [ ]:
# Upload the data file

file_content = """Category,Cost
Accommodation, 674.56
Transportation, 2301.00
Meals, 267.89
Misc., 34.50"""
file_name = "data.txt"

file = (file_name, file_content)

upload_result = agent_client.files.upload_and_poll(
    file=file, purpose=FilePurpose.AGENTS
)
upload_result

{'object': 'file', 'id': 'assistant-MFAiZciP4tzbdPSy1YsTdX', 'purpose': 'assistants', 'filename': 'data.txt', 'bytes': 86, 'created_at': 1751177528, 'expires_at': None, 'status': 'processed', 'status_details': None}

In [ ]:
# Create a CodeInterpreterTool
code_interpreter = CodeInterpreterTool(file_ids=[upload_result.id])

In [16]:
# Define an agent that uses the CodeInterpreterTool
agent = agent_client.create_agent(
     model=AGENT_MODEL_DEPLOYMENT,
     name="data-agent",
     instructions="You are an AI agent that analyzes the data in the file that has been uploaded. Use Python to calculate statistical metrics as necessary.",
     tools=code_interpreter.definitions,
     tool_resources=code_interpreter.resources,
)
print(f"Using agent: {agent.name}")

Using agent: data-agent


In [17]:
# Create a thread for the conversation
thread = agent_client.threads.create()

# Send a prompt to the agent
message = agent_client.messages.create(
     thread_id=thread.id,
     role="user",
     content="Can you help me analyze this data using Python code?",
 )

run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

In [18]:
# Check the run status for failures
if run.status == "failed":
     print(f"Run failed: {run.last_error}")

In [19]:
# Get the conversation history
print("\nConversation Log:\n")
messages = agent_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role}: {last_msg.text.value}\n")


Conversation Log:

MessageRole.USER: Can you help me analyze this data using Python code?

MessageRole.AGENT: Sure, I'd be happy to help you analyze the data. First, let's take a look at the contents of the uploaded file to understand its structure and the type of data it contains. I'll read the file and display its contents.

MessageRole.AGENT: The file appears to be a CSV (Comma-Separated Values) format containing two columns: "Category" and "Cost." Here's a quick overview of the data structure:

```
Category,Cost
Accommodation,674.56
Transportation,2301.00
Meals,267.89
Misc.,34.50
```

Next, let's read the entire CSV file into a DataFrame for analysis and calculate some statistical metrics. I'll do that now.

MessageRole.AGENT: The data has been successfully loaded into a DataFrame. Here's a summary of the contents:

### Data Overview:
|   | Category       | Cost   |
|---|----------------|--------|
| 0 | Accommodation  | 674.56 |
| 1 | Transportation | 2301.00|
| 2 | Meals         

In [3]:
# Clean up
agent_client.delete_agent(agent.id)